In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import json
import logging
from pathlib import Path
from typing import List, Tuple, Dict
import warnings
import numpy as np
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import (
XLMRobertaTokenizer,
XLMRobertaForSequenceClassification,
get_linear_schedule_with_warmup,
AutoModelForSequenceClassification,
AutoTokenizer
)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.model_selection import StratifiedKFold
import altair as alt

from src.baseline.baseline import train_df, SEED
from src.finetune.dataloader import MultilingualDataset, StratifiedMultilingualSplitter, DynamicUndersamplingSampler
from src.finetune.train import train_epoch, validate
from src.finetune.utils import calculate_metrics, calculate_weights
from src.finetune.finetuner import run_train, run_inference

warnings.filterwarnings("ignore")

2025-12-04 19:43:50.589620: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 19:43:50.620227: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 19:43:51.216850: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


✓ All random seeds set to 42
training files: ['train_en.csv', 'train_it.csv', 'train_es.csv']
Total training samples: 2988
CLASS DISTRIBUTION

Overall:
  Class 0 (NOT_RECLAMATORY): 2560 (85.7%)
  Class 1 (RECLAMATORY): 428 (14.3%)
  Total: 2988

Per Language:
  EN: Class 0=938, Class 1=88, Total=1026
  ES: Class 0=743, Class 1=133, Total=876
  IT: Class 0=879, Class 1=207, Total=1086




# Model Configuration

In [2]:
RUN_1 = "cardiffnlp/twitter-xlm-roberta-base"
RUN_2 = "../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model"  # resultant model of 6_Finetune-LM-with-Optuna-for-MLM.ipynb

RUN = RUN_2

class BaseConfig:
    MODEL_NAME = RUN  
    NUM_LABELS = 2
    MAX_LENGTH = 128
    NUM_FROZEN_LAYERS = 10      
    LEARNING_RATE = 5e-5
    WEIGHT_DECAY = 0.01
    BATCH_SIZE = 8
    NUM_EPOCHS = 10
    GRADIENT_ACCUMULATION_STEPS = 2
    WARMUP_RATIO = 0.15
    PATIENCE = 3
    EVAL_STRATEGY = "epoch"
    N_SPLITS = 5
    TRAIN_RATIO = 0.8
    VAL_RATIO = 0.2
    DYNAMIC_UNDERSAMPLE = False
    MAX_MODELS_TO_SAVE = 2
    OUTPUT_DIR = "../fine_tuned_models/mlm"
    RESULTS_DIR = "../results/fine_tuned_mlm/"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Setup

In [3]:
def setup_model(Config, num_frozen_layers: int = None):
    if num_frozen_layers is None:
        num_frozen_layers = Config.NUM_FROZEN_LAYERS
    
    tokenizer = XLMRobertaTokenizer.from_pretrained(Config.MODEL_NAME)
    model = XLMRobertaForSequenceClassification.from_pretrained(
        Config.MODEL_NAME, num_labels=Config.NUM_LABELS
    )
    
    model = model.to(Config.DEVICE)
    
    torch.nn.init.xavier_uniform_(model.classifier.dense.weight)
    torch.nn.init.zeros_(model.classifier.dense.bias)
    torch.nn.init.xavier_uniform_(model.classifier.out_proj.weight)
    torch.nn.init.zeros_(model.classifier.out_proj.bias)
    
    for param in model.roberta.embeddings.parameters():
        param.requires_grad = False
    
    num_total_layers = len(model.roberta.encoder.layer)
    for idx in range(min(num_frozen_layers, num_total_layers)):
        for param in model.roberta.encoder.layer[idx].parameters():
            param.requires_grad = False
    
    if num_frozen_layers >= num_total_layers and model.roberta.pooler is not None:
        for param in model.roberta.pooler.parameters():
            param.requires_grad = False
    
    for param in model.classifier.parameters():
        param.requires_grad = True
    
    print(f"Froze: Embeddings + First {min(num_frozen_layers, num_total_layers)} Encoder Layers")
    print("Trainable: Classification Head + Remaining Encoder Layers")
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(
        f"Trainable parameters: {trainable_params:,} / {total_params:,} ({100 * trainable_params / total_params:.2f}%)"
    )

    return model, tokenizer

# Model Storage Configuration

In [4]:
class ModelCheckpointManager:
    def init(self, max_models: int = 2, output_dir: str = "./models"):
        self.max_models = max_models
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.best_models = []

    def save_checkpoint(self, model, score: float, epoch: int, fold: int) -> bool:
        checkpoint_name = f"fold_{fold}_epoch_{epoch}_f1_{score:.4f}.pt"
        checkpoint_path = self.output_dir / checkpoint_name
    
        self.best_models.append((score, checkpoint_path, epoch, fold))
        self.best_models.sort(reverse=True, key=lambda x: x)
    
        if len(self.best_models) > self.max_models:
            worst_score, worst_path, worst_epoch, worst_fold = self.best_models.pop()
            if worst_path.exists():
                worst_path.unlink()
                logger.info(f"Deleted checkpoint: {worst_path}")
    
        torch.save(model.state_dict(), checkpoint_path)
        logger.info(
            f"Saved checkpoint: {checkpoint_path} (F1: {score:.4f}, Fold: {fold}, Epoch: {epoch})"
        )
    
        return True
    
    def get_best_models(self) -> List[Tuple]:
        return [(score, path, epoch, fold) for score, path, epoch, fold in self.best_models]

# Single Fold Training

In [5]:
def train_single_fold(train_df: pd.DataFrame, val_df: pd.DataFrame, Config, fold_id: int = 0, trial_id: int = 0):
    try:
        model, tokenizer = setup_model(Config, num_frozen_layers=Config.NUM_FROZEN_LAYERS)
    
        label_weights, language_weights, pos_weight = calculate_weights(train_df)
    
        train_dataset = MultilingualDataset(
            texts=train_df["text"].tolist(),
            labels=train_df["label"].tolist(),
            languages=train_df["lang"].tolist(),
            tokenizer=tokenizer,
            max_length=Config.MAX_LENGTH,
        )
    
        val_dataset = MultilingualDataset(
            texts=val_df["text"].tolist(),
            labels=val_df["label"].tolist(),
            languages=val_df["lang"].tolist(),
            tokenizer=tokenizer,
            max_length=Config.MAX_LENGTH,
        )
    
        optimizer = AdamW(
            model.parameters(),
            lr=Config.LEARNING_RATE,
            weight_decay=Config.WEIGHT_DECAY,
        )
    
        balanced_train_size = 3 * len(train_df[train_df["label"] == 1])
        total_steps = (
            balanced_train_size
            // (Config.BATCH_SIZE * Config.GRADIENT_ACCUMULATION_STEPS)
            * Config.NUM_EPOCHS
        )
        warmup_steps = int(Config.WARMUP_RATIO * total_steps)
    
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps,
        )
    
        best_val_f1 = 0
        patience_counter = 0
        fold_results = []
    
        for epoch in range(Config.NUM_EPOCHS):
            
            if Config.DYNAMIC_UNDERSAMPLE:
                undersampler = DynamicUndersamplingSampler(
                    train_df, minority_class=1, seed=SEED
                )
                balanced_indices = undersampler.get_balanced_indices(epoch)
                train_subset_df = train_df.iloc[balanced_indices].reset_index(drop=True)
            else:
                train_subset_df = train_df
    
            train_dataset_epoch = MultilingualDataset(
                texts=train_subset_df["text"].tolist(),
                labels=train_subset_df["label"].tolist(),
                languages=train_subset_df["lang"].tolist(),
                tokenizer=tokenizer,
                max_length=Config.MAX_LENGTH,
            )
    
            train_loader = DataLoader(
                train_dataset_epoch,
                batch_size=Config.BATCH_SIZE,
                shuffle=True,
            )
    
            val_loader = DataLoader(
                val_dataset,
                batch_size=Config.BATCH_SIZE,
                shuffle=False,
            )
    
            train_loss = train_epoch(
                model,
                train_loader,
                optimizer,
                scheduler,
                label_weights,
                language_weights,
                pos_weight,
                Config
            )
    
            val_loss, val_preds, val_labels, val_languages = validate(
                model, val_loader, Config
            )
    
            val_metrics = calculate_metrics(val_preds, val_labels, val_languages)
            current_f1 = val_metrics["overall"]["macro_f1"]
            
            print(f"Trial {trial_id}, Fold {fold_id}, Epoch {epoch+1}: F1={current_f1:.4f}")
    
            if current_f1 > best_val_f1:
                best_val_f1 = current_f1
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= Config.PATIENCE:
                    logger.info(f"Early stopping at epoch {epoch}")
                    break
    
            fold_results.append({
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_f1": current_f1
            })
    
        return best_val_f1, fold_results
    
    except Exception as e:
        print(f"Error in fold {fold_id}: {str(e)}")
        return 0.0, []

# Optuna Parameter Selection

In [6]:
def optuna_objective(trial, train_df: pd.DataFrame, base_config: BaseConfig):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_epochs", 5, 12)
    
    print(f"\n{'='*80}")
    print(f"Trial {trial.number}")
    print(f"{'='*80}")
    print(f"Learning Rate: {learning_rate:.2e}")
    print(f"Weight Decay: {weight_decay:.4f}")
    print(f"Batch Size: {batch_size}")
    print(f"Num Epochs: {num_epochs}")
    
    class TrialConfig(base_config.__class__):
        LEARNING_RATE = learning_rate
        WEIGHT_DECAY = weight_decay
        BATCH_SIZE = batch_size
        NUM_EPOCHS = num_epochs
        OUTPUT_DIR = f"{base_config.OUTPUT_DIR}/trial_{trial.number}"
    
    trial_config = TrialConfig()
    
    fold_scores = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
        
        fold_train = train_df.iloc[train_idx].reset_index(drop=True)
        fold_val = train_df.iloc[val_idx].reset_index(drop=True)
        
        print(f"\n  Fold {fold_idx+1}/5: Train={len(fold_train)}, Val={len(fold_val)}")
        
        try:
            fold_f1, fold_results = train_single_fold(
                fold_train, 
                fold_val, 
                trial_config, 
                fold_id=fold_idx,
                trial_id=trial.number
            )
            
            fold_scores.append(fold_f1)
            print(f"  Fold {fold_idx+1} F1: {fold_f1:.4f}")
            
            avg_so_far = np.mean(fold_scores)
            trial.report(avg_so_far, fold_idx)
            
            if trial.should_prune():
                print(f"  Trial pruned at fold {fold_idx}")
                raise optuna.TrialPruned()
                
        except optuna.TrialPruned:
            raise
        except Exception as e:
            logger.error(f"Fold {fold_idx} failed: {str(e)}")
            return 0.0
    
    avg_f1 = np.mean(fold_scores)
    print(f"\nTrial {trial.number} - Average F1: {avg_f1:.4f}")
    print(f"Fold scores: {[f'{f:.4f}' for f in fold_scores]}\n")
    
    return avg_f1

# Visulize Optuna Parameter Selection Results

In [7]:
def visualize_optuna_results(study, output_dir: str = "../figures"):
    os.makedirs(output_dir, exist_ok=True)

    # Prepare data
    trial_data = []
    for trial in study.trials:
        if trial.value is not None:
            trial_data.append({
                "trial": trial.number,
                "f1": trial.value,
                "lr": trial.params.get("learning_rate", 0),
                "batch_size": trial.params.get("batch_size", 0),
                "epochs": trial.params.get("num_epochs", 0),
                "weight_decay": trial.params.get("weight_decay", 0)
            })
    
    df = pd.DataFrame(trial_data)
    
    print("\nGenerating visualizations...")
    
    # Plot 1: Optimization History
    opt_history = alt.Chart(df).mark_line(point=True).encode(
        x=alt.X("trial:Q", title="Trial Number"),
        y=alt.Y("f1:Q", title="F1 Score"),
        tooltip=["trial:Q", alt.Tooltip("f1:Q", format=".4f")]
    ).properties(
        width=700, height=400, title="Optimization History: F1 Score Over Trials"
    ).interactive()
    
    opt_history_path = f"{output_dir}/optuna_optimization_history.svg"
    opt_history.save(opt_history_path)
    print(f"Saved: optuna_optimization_history.svg")
    
    # Plot 2: Learning Rate Impact
    lr_plot = alt.Chart(df).mark_circle(size=100).encode(
        x=alt.X("lr:Q", title="Learning Rate", scale=alt.Scale(type="log")),
        y=alt.Y("f1:Q", title="F1 Score"),
        color=alt.Color("epochs:O", title="Num Epochs"),
        tooltip=["trial:Q", alt.Tooltip("f1:Q", format=".4f"), "lr:Q", "epochs:O"]
    ).properties(
        width=600, height=400, title="Learning Rate vs F1 Score"
    ).interactive()
    
    lr_path = f"{output_dir}/optuna_learning_rate_impact.svg"
    lr_plot.save(lr_path)
    print(f"Saved: optuna_learning_rate_impact.svg")
    
    # Plot 3: Batch Size Impact
    bs_plot = alt.Chart(df).mark_boxplot().encode(
        x=alt.X("batch_size:O", title="Batch Size"),
        y=alt.Y("f1:Q", title="F1 Score"),
        tooltip=["trial:Q", alt.Tooltip("f1:Q", format=".4f")]
    ).properties(
        width=500, height=400, title="Batch Size vs F1 Score"
    )
    
    bs_path = f"{output_dir}/optuna_batch_size_impact.svg"
    bs_plot.save(bs_path)
    print(f"Saved: optuna_batch_size_impact.svg")
    
    # Plot 4: Weight Decay Impact
    wd_plot = alt.Chart(df).mark_circle(size=100).encode(
        x=alt.X("weight_decay:Q", title="Weight Decay"),
        y=alt.Y("f1:Q", title="F1 Score"),
        color=alt.Color("f1:Q", scale=alt.Scale(scheme="viridis")),
        tooltip=["trial:Q", alt.Tooltip("f1:Q", format=".4f"), "weight_decay:Q"]
    ).properties(
        width=600, height=400, title="Weight Decay vs F1 Score"
    ).interactive()
    
    wd_path = f"{output_dir}/optuna_weight_decay_impact.svg"
    wd_plot.save(wd_path)
    print(f"Saved: optuna_weight_decay_impact.svg")
    
    # Plot 5: Epochs Impact
    epochs_plot = alt.Chart(df).mark_boxplot().encode(
        x=alt.X("epochs:O", title="Number of Epochs"),
        y=alt.Y("f1:Q", title="F1 Score"),
        tooltip=["trial:Q", alt.Tooltip("f1:Q", format=".4f")]
    ).properties(
        width=600, height=400, title="Number of Epochs vs F1 Score"
    )
    
    epochs_path = f"{output_dir}/optuna_epochs_impact.svg"
    epochs_plot.save(epochs_path)
    print(f"Saved: optuna_epochs_impact.svg")
    
    # Save trials dataframe
    trials_csv = f"{output_dir}/optuna_trials.csv"
    df.to_csv(trials_csv, index=False)
    print(f"Saved: optuna_trials.csv")
    
    print(f"\nAll visualizations saved to: {output_dir}\n")
    
    return df

In [8]:
def run_optuna_optimization(train_df: pd.DataFrame, n_trials: int = 10):
    base_config = BaseConfig()

    print("\n" + " "*80)
    print("STEP 1: OPTUNA HYPERPARAMETER OPTIMIZATION")
    print(" "*80)
    print(f"Number of trials: {n_trials}")
    print(f"CV folds per trial: 5")
    print(f"Total model trainings: {n_trials * 5}")
    print(" "*80 + "\n")
    
    sampler = TPESampler(seed=SEED)
    pruner = MedianPruner(n_startup_trials=2, n_warmup_steps=10)
    
    study = optuna.create_study(
        direction="maximize",
        sampler=sampler,
        pruner=pruner,
        study_name="classification_optimization"
    )
    
    study.optimize(
        lambda trial: optuna_objective(trial, train_df, base_config),
        n_trials=n_trials,
        gc_after_trial=True,
        show_progress_bar=False
    )
    
    best_trial = study.best_trial
    
    print("\n" + "="*80)
    print("BEST TRIAL")
    print("="*80)
    print(f"Trial: {best_trial.number}")
    print(f"Best F1 Score: {best_trial.value:.4f}")
    print(f"\nBest Hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")
    print("="*80 + "\n")
    
    # Visualize
    print("\n" + " "*80)
    print(" STEP 2: VISUALIZE RESULTS")
    print(" "*80 + "\n")
    
    trial_df = visualize_optuna_results(study)
    
    print(f"Top 5 Trials:")
    print(trial_df.nlargest(5, "f1")[["trial", "f1", "lr", "batch_size", "epochs"]])
    
    return study, best_trial

# Train Final Model with Best Parameters

In [9]:
def train_final_model(best_trial, train_df: pd.DataFrame, base_config: BaseConfig):
    print("\n" + " "*80)
    print("STEP 3: TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS")
    print(" "*80)
    
    class FinalConfig(BaseConfig):
        LEARNING_RATE = best_trial.params["learning_rate"]
        WEIGHT_DECAY = best_trial.params["weight_decay"]
        BATCH_SIZE = best_trial.params["batch_size"]
        NUM_EPOCHS = best_trial.params["num_epochs"]
        OUTPUT_DIR = f"{base_config.OUTPUT_DIR}/final_model"
        RESULTS_DIR = f"{base_config.RESULTS_DIR}final_model/"
    
    final_config = FinalConfig()
    
    print(f"\nFinal Training Configuration:")
    print(f"Learning Rate: {final_config.LEARNING_RATE:.2e}")
    print(f"Weight Decay: {final_config.WEIGHT_DECAY:.4f}")
    print(f"Batch Size: {final_config.BATCH_SIZE}")
    print(f"Num Epochs: {final_config.NUM_EPOCHS}\n")
    
    final_model_path = run_train(train_df, final_config)
    
    return final_config, final_model_path

# Inference

In [10]:
class InferenceConfig:
    MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base"
    NUM_LABELS = 2
    MAX_LENGTH = 128
    BATCH_SIZE = 32
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    CHECKPOINT_PATH = None # Will be set from best model

class TestDataset(Dataset):
    def __init__(self, texts: List[str], tokenizer, max_length: int = 128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }


def run_inference(df: pd.DataFrame, config: InferenceConfig) -> Dict:
    print(f"Running inference on {len(df)} samples...")
    print(f"Device: {config.DEVICE}")
    
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
    
    model = AutoModelForSequenceClassification.from_pretrained(
        config.MODEL_NAME,
        num_labels=config.NUM_LABELS
    )
    
    checkpoint = torch.load(config.CHECKPOINT_PATH, map_location=config.DEVICE)
    
    if isinstance(checkpoint, dict):
        if "model_state_dict" in checkpoint:
            state_dict = checkpoint["model_state_dict"]
        elif "state_dict" in checkpoint:
            state_dict = checkpoint["state_dict"]
        else:
            state_dict = checkpoint
    else:
        state_dict = checkpoint
    
    model.load_state_dict(state_dict, strict=False)
    print(f"Loaded checkpoint from: {config.CHECKPOINT_PATH}")
    
    model.to(config.DEVICE)
    model.eval()
    
    texts = df["text"].tolist()
    labels = df["label"].tolist()
    languages = df["lang"].tolist()
    
    dataset = TestDataset(texts, tokenizer, config.MAX_LENGTH)
    dataloader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=False)
    
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Inference"):
            input_ids = batch["input_ids"].to(config.DEVICE)
            attention_mask = batch["attention_mask"].to(config.DEVICE)
    
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
    
            probs = torch.softmax(logits, dim=-1)
            preds = torch.argmax(logits, dim=-1)
    
            all_probs.extend(probs.cpu().numpy().tolist())
            all_preds.extend(preds.cpu().numpy().tolist())
    
    metrics = calculate_metrics(all_preds, labels, languages)
    
    print("\n" + "="*80)
    print("INFERENCE RESULTS ON TRAINING DATA")
    print("="*80)
    
    print(f"\nOverall Metrics:")
    print(f"Macro Precision:{metrics['overall']['macro_precision']:.4f}")
    print(f"Macro Recall:{metrics['overall']['macro_recall']:.4f}")
    print(f"Macro F1:{metrics['overall']['macro_f1']:.4f}")
    
    print(f"\nPer-Language Metrics:")
    for lang in sorted([k for k in metrics.keys() if k != "overall"]):
        print(f"{lang.upper()}:")
        print(f"Precision:{metrics[lang]['macro_precision']:.4f}")
        print(f"Recall:{metrics[lang]['macro_recall']:.4f}")
        print(f"F1:{metrics[lang]['macro_f1']:.4f}")
    
    print("="*80 + "\n")
    
    return {
        "predictions": all_preds,
        "probabilities": all_probs,
        "labels": labels,
        "languages": languages,
        "metrics": metrics
    }


# Actual Run

### Load Data

In [11]:
original_data = train_df
augmented_data = pd.read_csv("../data/augmented_multilingual_tweets.csv")
print(original_data.shape, augmented_data.shape)

(2988, 5) (5976, 8)


In [12]:
merged_data = pd.concat([original_data, augmented_data[list(original_data.columns)]], ignore_index=True)
print(merged_data.shape)

(8964, 5)


### Optuna Parameter Selection

In [13]:
base_config = BaseConfig()
study, best_trial = run_optuna_optimization(merged_data, n_trials=10)

[I 2025-12-04 19:44:58,211] A new study created in memory with name: classification_optimization



                                                                                
STEP 1: OPTUNA HYPERPARAMETER OPTIMIZATION
                                                                                
Number of trials: 10
CV folds per trial: 5
Total model trainings: 50
                                                                                


Trial 0
Learning Rate: 4.33e-05
Weight Decay: 0.0951
Batch Size: 8
Num Epochs: 6

  Fold 1/5: Train=7171, Val=1793


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 19:44:59,762 - INFO - Label weights: {0: 0.5835774739583334, 1: 3.491236611489776}
2025-12-04 19:44:59,762 - INFO - Language weights: {'es': 0.9847470941344993, 'en': 1.0008296471794096, 'it': 1.0144232586860908}
2025-12-04 19:44:59,762 - INFO - Pos weight (for BCE): 5.9825


Froze: Embeddings + First 10 Encoder Layers
Trainable: Classification Head + Remaining Encoder Layers
Trainable parameters: 14,767,874 / 278,045,186 (5.31%)


Validating: 100%|██████████| 225/225 [00:02<00:00, 109.99it/s]


...

Trial 9, Fold 4, Epoch 5: F1=0.6821


Validating: 100%|██████████| 112/112 [00:01<00:00, 69.53it/s]


Trial 9, Fold 4, Epoch 6: F1=0.7076


Validating: 100%|██████████| 112/112 [00:01<00:00, 69.49it/s]
[I 2025-12-04 20:51:57,842] Trial 9 finished with value: 0.7174014345909159 and parameters: {'learning_rate': 0.0001037084466895453, 'weight_decay': 0.09218742350231168, 'batch_size': 16, 'num_epochs': 7}. Best is trial 2 with value: 0.7918504359347013.


Trial 9, Fold 4, Epoch 7: F1=0.6957
  Fold 5 F1: 0.7076

Trial 9 - Average F1: 0.7174
Fold scores: ['0.7351', '0.7500', '0.6982', '0.6962', '0.7076']


BEST TRIAL
Trial: 2
Best F1 Score: 0.7919

Best Hyperparameters:
  learning_rate: 0.00025959425503112657
  weight_decay: 0.021233911067827616
  batch_size: 32
  num_epochs: 9


                                                                                
 STEP 2: VISUALIZE RESULTS
                                                                                


Generating visualizations...
✓ Saved: optuna_optimization_history.svg
✓ Saved: optuna_learning_rate_impact.svg
✓ Saved: optuna_batch_size_impact.svg
✓ Saved: optuna_weight_decay_impact.svg
✓ Saved: optuna_epochs_impact.svg
✓ Saved: optuna_trials.csv

All visualizations saved to: ../figures

Top 5 Trials:
   trial        f1        lr  batch_size  epochs
2      2  0.791850  0.000260          32       9
9      9  0.717401  0.000104          16       7
3      3  0.648432  0.00005

### Final Model Training w.r.t Best Parameters

In [14]:
final_config, final_model_path = train_final_model(best_trial, augmented_data, base_config)

2025-12-04 20:53:11,567 - INFO - Fold 0: Train=3824, Val=956
2025-12-04 20:53:11,567 - INFO -   Train label dist: {0: 3276, 1: 548}
2025-12-04 20:53:11,568 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 20:53:11,573 - INFO - Fold 1: Train=3824, Val=957
2025-12-04 20:53:11,574 - INFO -   Train label dist: {0: 3277, 1: 547}
2025-12-04 20:53:11,575 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 20:53:11,581 - INFO - Fold 2: Train=3824, Val=957
2025-12-04 20:53:11,582 - INFO -   Train label dist: {0: 3277, 1: 547}
2025-12-04 20:53:11,583 - INFO -   Train lang dist: {'es': 1352, 'en': 1256, 'it': 1216}
2025-12-04 20:53:11,589 - INFO - Fold 3: Train=3824, Val=957
2025-12-04 20:53:11,589 - INFO -   Train label dist: {0: 3278, 1: 546}
2025-12-04 20:53:11,590 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 20:53:11,594 - INFO - Fold 4: Train=3824, Val=957
2025-12-04 20:53:11,594 - INFO -   Train label dist: {0


                                                                                
STEP 3: TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS
                                                                                

Final Training Configuration:
  Learning Rate: 2.60e-04
  Weight Decay: 0.0212
  Batch Size: 32
  Num Epochs: 9


FOLD 0:
  Train: 548 positive samples
  Val:   137 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 20:53:12,316 - INFO - Froze: Embeddings + First 10 Encoder Layers
2025-12-04 20:53:12,317 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 20:53:12,317 - INFO - Trainable parameters: 14,767,874 / 278,045,186 (5.31%)
2025-12-04 20:53:12,318 - INFO - Label weights: {0: 0.5836385836385837, 1: 3.489051094890511}
2025-12-04 20:53:12,318 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 20:53:12,318 - INFO - Pos weight (for BCE): 5.9781
2025-12-04 20:53:12,319 - INFO - 
E


FOLD 1:
  Train: 547 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 20:54:04,959 - INFO - Froze: Embeddings + First 10 Encoder Layers
2025-12-04 20:54:04,960 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 20:54:04,960 - INFO - Trainable parameters: 14,767,874 / 278,045,186 (5.31%)
2025-12-04 20:54:04,962 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 20:54:04,962 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 20:54:04,962 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 20:54:04,963 - INFO - 
E

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 20:54:56,193 - INFO - Froze: Embeddings + First 10 Encoder Layers
2025-12-04 20:54:56,193 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 20:54:56,194 - INFO - Trainable parameters: 14,767,874 / 278,045,186 (5.31%)
2025-12-04 20:54:56,195 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 20:54:56,195 - INFO - Language weights: {'es': 0.9409476243674836, 'en': 1.0128671880134061, 'it': 1.0461851876191102}
2025-12-04 20:54:56,195 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 20:54:56,196 - INFO - 
E


FOLD 3:
  Train: 546 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 20:55:35,232 - INFO - Froze: Embeddings + First 10 Encoder Layers
2025-12-04 20:55:35,233 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 20:55:35,233 - INFO - Trainable parameters: 14,767,874 / 278,045,186 (5.31%)
2025-12-04 20:55:35,234 - INFO - Label weights: {0: 0.5832824893227577, 1: 3.501831501831502}
2025-12-04 20:55:35,235 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 20:55:35,235 - INFO - Pos weight (for BCE): 6.0037
2025-12-04 20:55:35,236 - INFO - 
E


FOLD 4:
  Train: 547 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 20:56:26,252 - INFO - Froze: Embeddings + First 10 Encoder Layers
2025-12-04 20:56:26,252 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 20:56:26,253 - INFO - Trainable parameters: 14,767,874 / 278,045,186 (5.31%)
2025-12-04 20:56:26,254 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 20:56:26,254 - INFO - Language weights: {'es': 0.9416953224870754, 'en': 1.0129222776114961, 'it': 1.045382399901429}
2025-12-04 20:56:26,254 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 20:56:26,255 - INFO - 
Ep

### Inference with Peformance Metrics

In [15]:
inference_config = InferenceConfig()
inference_config.CHECKPOINT_PATH = str(final_model_path)

inference_results = run_inference(merged_data, inference_config)

Running inference on 8964 samples...
Device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded checkpoint from: ../fine_tuned_models/mlm/final_model/checkpoints/fold_0_epoch_7_f1_0.7841.pt


Inference: 100%|██████████| 281/281 [00:07<00:00, 38.21it/s]



INFERENCE RESULTS ON TRAINING DATA

Overall Metrics:
  Macro Precision: 0.8027
  Macro Recall:    0.7472
  Macro F1:        0.7706

Per-Language Metrics:
  EN:
    Precision: 0.7634
    Recall:    0.7952
    F1:        0.7777
  ES:
    Precision: 0.8446
    Recall:    0.8117
    F1:        0.8268
  IT:
    Precision: 0.8677
    Recall:    0.6347
    F1:        0.6785



In [16]:
print(f"\nOptuna Trials: {len(study.trials)}")
print(f"Best Trial: {best_trial.number}")
print(f"Best F1 Score (CV): {best_trial.value:.4f}")
print(f"Final Inference F1: {inference_results['metrics']['overall']['macro_f1']:.4f}")
print(f"\nBest Hyperparameters:")
for key, val in best_trial.params.items():
    print(f"  {key}: {val}")
print(f"\nModel saved at: {final_config.OUTPUT_DIR}")
print(f"Results saved at: {final_config.RESULTS_DIR}")


Optuna Trials: 10
Best Trial: 2
Best F1 Score (CV): 0.7919
Final Inference F1: 0.7706

Best Hyperparameters:
  learning_rate: 0.00025959425503112657
  weight_decay: 0.021233911067827616
  batch_size: 32
  num_epochs: 9

Model saved at: ../fine_tuned_models/mlm/final_model
Results saved at: ../results/fine_tuned_mlm/final_model/


# Prediction Threshold Validation

In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [18]:
probs = np.array(inference_results['probabilities'])  # Shape: (N, 2)
probs_class_1 = probs[:, 1]  # Get positive class probabilities
labels = np.array(inference_results['labels'])
languages = inference_results['languages']

print(f"Probability stats:")
print(f"Min:{probs_class_1.min():.4f}")
print(f"Max:{probs_class_1.max():.4f}")
print(f"Mean:{probs_class_1.mean():.4f}")
print(f"Median:{np.median(probs_class_1):.4f}")

results = []

for threshold in np.arange(0.1, 1.0, 0.1):
    # Apply threshold
    preds = (probs_class_1 >= threshold).astype(int)
    
    # Calculate metrics
    f1 = f1_score(labels, preds, average="macro", zero_division=0)
    precision = precision_score(labels, preds, average="macro", zero_division=0)
    recall = recall_score(labels, preds, average="macro", zero_division=0)
    
    results.append({
        "threshold": threshold,
        "f1": f1,
        "precision": precision,
        "recall": recall
    })

threshold_df = pd.DataFrame(results)
print(threshold_df.to_string(index=False))


print("\n" + "="*80)
print("LANGUAGE-SPECIFIC OPTIMAL THRESHOLDS")
print("="*80)

language_optimal_thresholds = {}

for lang in sorted(set(languages)):
    lang_mask = np.array(languages) == lang
    lang_probs = probs_class_1[lang_mask]
    lang_labels = np.array(labels)[lang_mask]
    
    best_threshold = None
    best_f1 = 0
    best_precision = 0
    best_recall = 0
    
    # Test all thresholds for this language
    for threshold in np.arange(0.05, 1.0, 0.05):
        preds = (lang_probs >= threshold).astype(int)
        f1 = f1_score(lang_labels, preds, average="macro", zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision_score(lang_labels, preds, average="macro", zero_division=0)
            best_recall = recall_score(lang_labels, preds, average="macro", zero_division=0)
    
    language_optimal_thresholds[lang] = best_threshold
    
    print(f"\n{lang.upper()}:")
    print(f"Optimal Threshold: {best_threshold:.2f}")
    print(f"F1 Score: {best_f1:.4f}")
    print(f"Precision: {best_precision:.4f}")
    print(f"Recall: {best_recall:.4f}")

print("\n" + "="*80)
print("APPLYING LANGUAGE-SPECIFIC THRESHOLDS")
print("="*80)

language_specific_preds = np.zeros_like(probs_class_1, dtype=int)

for lang in sorted(set(languages)):
    lang_mask = np.array(languages) == lang
    threshold = language_optimal_thresholds[lang]
    language_specific_preds[lang_mask] = (probs_class_1[lang_mask] >= threshold).astype(int)

print("\nOVERALL METRICS COMPARISON:")
print("-" * 80)

default_preds = (probs_class_1 >= 0.5).astype(int)
default_f1 = f1_score(labels, default_preds, average="macro", zero_division=0)
default_precision = precision_score(labels, default_preds, average="macro", zero_division=0)
default_recall = recall_score(labels, default_preds, average="macro", zero_division=0)

langspec_f1 = f1_score(labels, language_specific_preds, average="macro", zero_division=0)
langspec_precision = precision_score(labels, language_specific_preds, average="macro", zero_division=0)
langspec_recall = recall_score(labels, language_specific_preds, average="macro", zero_division=0)

print(f"\nDefault Threshold (0.5):")
print(f"F1: {default_f1:.4f}")
print(f"Precision: {default_precision:.4f}")
print(f"Recall: {default_recall:.4f}")

print(f"\nLanguage-Specific Thresholds:")
for lang in sorted(set(languages)):
    print(f"  {lang.upper()}: {language_optimal_thresholds[lang]:.2f}", end="")
print()

print(f"F1: {langspec_f1:.4f}")
print(f"Precision: {langspec_precision:.4f}")
print(f"Recall: {langspec_recall:.4f}")

print(f"\nOverall Improvement:")
print(f"F1 Change: {langspec_f1 - default_f1:+.4f} ({(langspec_f1/default_f1 - 1)*100:+.1f}%)")
print(f"Precision Change: {langspec_precision - default_precision:+.4f}")
print(f"Recall Change: {langspec_recall - default_recall:+.4f}")

print("\n" + "="*80)
print("PER-LANGUAGE COMPARISON")
print("="*80)

for lang in sorted(set(languages)):
    lang_mask = np.array(languages) == lang
    lang_labels = np.array(labels)[lang_mask]
    
    default_lang_preds = default_preds[lang_mask]
    default_lang_f1 = f1_score(lang_labels, default_lang_preds, average="macro", zero_division=0)
    default_lang_precision = precision_score(lang_labels, default_lang_preds, average="macro", zero_division=0)
    default_lang_recall = recall_score(lang_labels, default_lang_preds, average="macro", zero_division=0)
    
    langspec_lang_preds = language_specific_preds[lang_mask]
    langspec_lang_f1 = f1_score(lang_labels, langspec_lang_preds, average="macro", zero_division=0)
    langspec_lang_precision = precision_score(lang_labels, langspec_lang_preds, average="macro", zero_division=0)
    langspec_lang_recall = recall_score(lang_labels, langspec_lang_preds, average="macro", zero_division=0)
    
    print(f"\n{lang.upper()}:")
    print(f"Default (0.5):")
    print(f"F1: {default_lang_f1:.4f}, Precision: {default_lang_precision:.4f}, Recall: {default_lang_recall:.4f}")
    print(f"Language-Specific ({language_optimal_thresholds[lang]:.2f}):")
    print(f"F1: {langspec_lang_f1:.4f}, Precision: {langspec_lang_precision:.4f}, Recall: {langspec_lang_recall:.4f}")
    print(f"Improvement: {langspec_lang_f1 - default_lang_f1:+.4f} ({(langspec_lang_f1/default_lang_f1 - 1)*100:+.1f}%)")

print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORT - LANGUAGE-SPECIFIC THRESHOLDS")
print("="*80)

print("\nOverall Classification Report:")
print(classification_report(labels, language_specific_preds, target_names=['Class 0', 'Class 1']))

print("\nPer-Language Classification Reports:")
for lang in sorted(set(languages)):
    lang_mask = np.array(languages) == lang
    lang_labels = np.array(labels)[lang_mask]
    lang_preds = language_specific_preds[lang_mask]
    
    print(f"\n{lang.upper()} (Threshold: {language_optimal_thresholds[lang]:.2f}):")
    print(classification_report(lang_labels, lang_preds, target_names=['Class 0', 'Class 1']))

results_df = pd.DataFrame({
    'language': list(language_optimal_thresholds.keys()),
    'optimal_threshold': list(language_optimal_thresholds.values())
})

print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)
print(results_df.to_string(index=False))

Probability stats:
Min:0.0010
Max:0.9996
Mean:0.2448
Median:0.1993
 threshold       f1  precision   recall
       0.1 0.417996   0.598311 0.664659
       0.2 0.574726   0.628261 0.761282
       0.3 0.690419   0.674178 0.810448
       0.4 0.759620   0.734950 0.798832
       0.5 0.770560   0.802662 0.747208
       0.6 0.722379   0.852444 0.675470
       0.7 0.632847   0.870776 0.599615
       0.8 0.564557   0.882454 0.554968
       0.9 0.511879   0.902955 0.525441

LANGUAGE-SPECIFIC OPTIMAL THRESHOLDS

EN:
Optimal Threshold: 0.50
F1 Score: 0.7777
Precision: 0.7634
Recall: 0.7952

ES:
Optimal Threshold: 0.45
F1 Score: 0.8314
Precision: 0.8177
Recall: 0.8473

IT:
Optimal Threshold: 0.35
F1 Score: 0.7710
Precision: 0.7807
Recall: 0.7624

APPLYING LANGUAGE-SPECIFIC THRESHOLDS

OVERALL METRICS COMPARISON:
--------------------------------------------------------------------------------

Default Threshold (0.5):
F1: 0.7706
Precision: 0.8027
Recall: 0.7472

Language-Specific Thresholds:
  EN: 0.

# Submission

In [19]:
from src.finetune.finetuner import run_test

In [20]:
class InferenceConfig:
    MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base"
    NUM_LABELS = 2
    CHECKPOINT_PATH = "../fine_tuned_models/mlm/final_model/checkpoints/fold_0_epoch_8_f1_0.7845.pt"
    
    # YOUR OPTIMAL THRESHOLDS
    LANGUAGE_THRESHOLDS = {
        'en': 0.50,
        'es': 0.50,
        'it': 0.50
    }

    MAX_LENGTH = 128
    BATCH_SIZE = 32
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class InferenceConfigThreshold:
    MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base"
    NUM_LABELS = 2
    # CHECKPOINT_PATH = "../fine_tuned_models/base/final_model/checkpoints/fold_0_epoch_8_f1_0.7829.pt"
    CHECKPOINT_PATH = "../fine_tuned_models/mlm/final_model/checkpoints/fold_0_epoch_8_f1_0.7845.pt"
    
    # YOUR OPTIMAL THRESHOLDS
    LANGUAGE_THRESHOLDS = {
        'en': 0.50,
        'es': 0.45,
        'it': 0.35
    }
    MAX_LENGTH = 128
    BATCH_SIZE = 32
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
test_root = "../data/test_sets/Test Set/"
test_files = [file for file in os.listdir(test_root) if (file.endswith(".csv") and ("test" in file))]
print(f"training files: {test_files}")

test_df = pd.DataFrame()


for file in test_files:
    temp_df = pd.read_csv(os.path.join(test_root, file))
    if "en" in file:
        temp_df["bio"] = [None] * temp_df.shape[0]
    test_df = pd.concat([test_df, temp_df], ignore_index=True)

training files: ['es_test.csv', 'it_test.csv', 'en_test.csv']


In [22]:
test_df.shape

(1995, 4)

In [23]:
results = run_test(test_df, InferenceConfig)

Running inference on 1995 samples...
Device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded checkpoint from: ../fine_tuned_models/mlm/final_model/checkpoints/fold_0_epoch_8_f1_0.7845.pt


Inference: 100%|██████████| 63/63 [00:01<00:00, 39.00it/s]


In [24]:
submission_folder = "../submissions/"
os.makedirs(submission_folder, exist_ok=True)
submission_file = os.path.join(submission_folder, "multipride2025_KIT-TIP-NLP_2.tsv")
final_submission = dict()
final_submission["id"] = results["ids"]
final_submission["label"] = results["predictions"]
final_submission["lang"] = results["languages"]


final_submission = pd.DataFrame.from_dict(final_submission)
print(final_submission.shape)
final_submission.to_csv(submission_file, sep="\t", index=False)

(1995, 3)


### For ablation Study 

In [25]:
results = run_test(test_df, InferenceConfigThreshold)

Running inference on 1995 samples...
Device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded checkpoint from: ../fine_tuned_models/mlm/final_model/checkpoints/fold_0_epoch_8_f1_0.7845.pt


Inference: 100%|██████████| 63/63 [00:01<00:00, 39.00it/s]


In [26]:
submission_file = os.path.join(submission_folder, "multipride2025_KIT-TIP-NLP_4.tsv")
final_submission = dict()
final_submission["id"] = results["ids"]
final_submission["label"] = results["predictions"]
final_submission["lang"] = results["languages"]


final_submission = pd.DataFrame.from_dict(final_submission)
print(final_submission.shape)
final_submission.to_csv(submission_file, sep="\t", index=False, )

(1995, 3)
